In [3]:
import cv2
import argparse
import numpy as np
import utils
import os 
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import gc

In [ ]:
%%time
utils.extract_frames("/data/TrainSet/Drone1/Morning/1.1.1.mov", out_dir='/data/out', save=True, skip=200)

In [2]:
%%time
# read input image
list_images = list()
list_images = utils.get_files_by_ext("/data/out/", "jpg")
print("We got ", len(list_images), " images")

We got  28  images
CPU times: user 2.13 ms, sys: 739 µs, total: 2.87 ms
Wall time: 9.05 ms


In [3]:
%%time
classes_file = "object-detection-opencv/yolov3.txt"
weights_file = "object-detection-opencv/yolov3-tiny.weights"
conf_file = "object-detection-opencv/yolov3-tiny.cfg"

conf_threshold = 0.5
nms_threshold = 0.4

# for image_path in list_images:
image = cv2.imread(list_images[0])

Width = image.shape[1]
Height = image.shape[0]

scale = 0.00392

# read class names from text file
classes = None
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# read pre-trained model and config file
# reads the weights and config file and creates the network.
# net = cv2.dnn.readNet(weights_file, conf_file)
model_path = "object-detection-opencv/yolov3-tiny.h5"
nb_out_layer = 3
model = tf.keras.models.load_model(model_path)

CPU times: user 820 ms, sys: 106 ms, total: 925 ms
Wall time: 1.61 s


In [5]:
%%time
batch_size = 10
batch_images = list()
for i in range(len(list_images)):
# for i in range(0, 10):
    batch_images.append(list_images[i])
    if i == len(list_images)-1 or len(batch_images)%batch_size==0:      
        
        time_min_blobing = time.time()
        # create input blob 
        # prepares the input image to run through the deep neural network
        blob = utils.image_to_matrix(batch_images)
        blob = blob.transpose((0,2,3,1))
        
        # set input blob for the network
#         net.setInput(blob)
        time_max_blobing = time.time()
        elapsed_time_blobing = time_max_blobing - time_min_blobing
        print("The blobing phase took ", round(elapsed_time_blobing,2), "s to be executed")
        print("\n")
        
        
        time_min_detection = time.time()
        # run inference through the network
        # and gather predictions from output layers
#         outs = net.forward(utils.get_output_layers(net))
        outs = model.predict(blob)

        # identify objects into the list of images analyzed by the network
        dict_obj_detected = {}
        dict_obj_detected = utils.detect_object(outs, batch_images, Width, Height, nb_out_layer)

        # identify pictures with at least one perso
        dict_danger = utils.detect_danger(dict_obj_detected)
        for image_name, is_dangerous in dict_danger.items():
            print("Is", image_name, "dangerous ? \t\t\t" , is_dangerous)
        time_max_detection = time.time()
        elapsed_time_detection = time_max_detection - time_min_detection
        print("The detection took ", round(elapsed_time_detection,2), "s to be executed")
        
        time_min_draw_bbox = time.time()
        # Draw bounding boxes and save images
        for image_path, image_items in dict_obj_detected.items():
            utils.get_bounding_box(image_path, image_items, \
                                   classes, COLORS, conf_threshold, nms_threshold
                                  )
        time_max_draw_bbox = time.time()
        elapsed_time_draw_bbox = time_max_draw_bbox - time_min_detection
        print("The draw of the bounding boxes and the image save took ", round(elapsed_time_draw_bbox,2), "s to be executed")         
        print("---------------------------------------------------------------------")
        
        batch_images = list()
    


The blobing phase took  0.21 s to be executed


Is /data/out/ObamaWalk.mp4_frame00399.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00400.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00401.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00402.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00403.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00404.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00405.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00406.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00407.jpg dangerous ? 			 True
Is /data/out/ObamaWalk.mp4_frame00408.jpg dangerous ? 			 True
The detection took  1.61 s to be executed
The draw of the bounding boxes and the image save took  2.23 s to be executed
---------------------------------------------------------------------
The blobing phase took  0.24 s to be executed


Is /data/out/ObamaWalk.mp4_frame00409.jpg dangerous ? 			 True
Is /data/out/ObamaWal

In [ ]:
utils.convert_frames_to_video("/data/out/output_with_bounding_box/", \
                              "/data/out/output_video/1.1.1_withBB.mp4", \
                              24, (Width,Height))